In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
data=pd.read_csv(r'../input/logisticregression-telecomcustomer-churmprediction/churn_data.csv')

In [ ]:
data1=pd.read_csv(r'../input/logisticregression-telecomcustomer-churmprediction/customer_data.csv')
data2=pd.read_csv(r'../input/logisticregression-telecomcustomer-churmprediction/internet_data.csv')

In [ ]:
churn=pd.DataFrame(data)
customer=pd.DataFrame(data1)
internet=pd.DataFrame(data2)

In [ ]:
churn.head()

In [ ]:
customer.head()

In [ ]:
internet.head()

In [ ]:
churn.shape

In [ ]:
customer.shape

In [ ]:
internet.shape

In [ ]:
customer.isnull().sum()

In [ ]:
churn.isnull().sum()

In [ ]:
internet.isnull().sum()

In [ ]:
merged1=pd.merge(customer,internet)

In [ ]:
final=pd.merge(churn,merged1)

In [ ]:
final.head()

In [ ]:
final.shape

In [ ]:
final.dtypes

In [ ]:
final.head()

In [ ]:
sns.countplot(final['Churn'])

Less than 50% of people are continuing with the services

In [ ]:
categorical_features=[feature for feature in final.columns if final[feature].dtype =='object']

In [ ]:
len(categorical_features)


In [ ]:
%matplotlib inline
for feature in categorical_features:
    if feature not in ['customerID','TotalCharges','Churn']:
        plt.figure(figsize=(18,6))
        sns.countplot(final[feature],hue=final['Churn'])
        plt.title(feature)
        plt.show()

In [ ]:
internet_services=['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']

# Feature Engineering

In [ ]:
data=final.copy()
for feature in internet_services:
    data[feature]=data[feature].replace({'No internet service':'No'})

In [ ]:
data=data.iloc[:,1:]

In [ ]:
data.head()

In [ ]:
data['MultipleLines']=data['MultipleLines'].replace({'No phone service':'No'})

In [ ]:
sns.kdeplot(data['tenure'])

In [ ]:
sns.kdeplot(data['MonthlyCharges'])

In [ ]:
ind=data[data['TotalCharges']==" "].index

In [ ]:
data=data.drop(index=ind)

In [ ]:
data.head()

In [ ]:
data['TotalCharges']=data['TotalCharges'].apply(lambda x: float(x) )

In [ ]:
sns.kdeplot(data['TotalCharges'])

In [ ]:
numerical_features=['Churn','MonthlyCharges','TotalCharges']

In [ ]:
sns.boxplot(data['MonthlyCharges'])

In [ ]:
sns.boxplot(data['TotalCharges'])

In [ ]:
sns.boxplot(data['tenure'])

# One Hot Encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
oe=OneHotEncoder()

In [ ]:
categorical_features.remove('customerID')
categorical_features.remove('TotalCharges')
categorical_features.remove('Churn')

In [ ]:
#data1=oe.fit_transform(data[categorical_features])

In [ ]:
data1=pd.get_dummies(data[categorical_features])

In [ ]:
#data1=pd.DataFrame.sparse.from_spmatrix(data1)

In [ ]:
data1.head()

In [ ]:
len(categorical_features)

In [ ]:
data.shape

In [ ]:
numerical_features=['tenure','MonthlyCharges','TotalCharges','SeniorCitizen','Churn']

In [ ]:
data2=data1.merge(data[numerical_features],left_index=True, right_index=True)

In [ ]:
data2.head()

In [ ]:
numerical_features.remove('SeniorCitizen')

In [ ]:
sns.heatmap(data2[numerical_features].corr())
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data2['Churn']=np.where(data2['Churn']=='No',0,1)

In [ ]:
data2.head()

In [ ]:
X=data2.iloc[:,:-1]

In [ ]:
X.head()

In [ ]:
Y=data2['Churn']

In [ ]:
ss=StandardScaler()

In [ ]:
col=[colu for colu in data.columns if colu not in numerical_features]

In [ ]:
X.head()

In [ ]:
#data3 = pd.concat([X[col].reset_index(drop=True),
                    #pd.DataFrame(ss.fit_transform(X[numerical_features]), columns=numerical_features)],
                    #axis=1)

In [ ]:
from math import sqrt
sns.boxplot(np.log(X['TotalCharges']))

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
Norm=Normalizer()

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=7)

In [ ]:
X_train.head()

# Support Vector Machine

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, Y_train)

In [ ]:
y_pred = svclassifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test,y_pred))
print(classification_report(Y_test,y_pred))

In [ ]:
Y_test.value_counts()

# XGBOOST

In [ ]:
estimate=data['Churn'].value_counts()['No']/data['Churn'].value_counts()['Yes']

In [ ]:
from numpy import mean
from xgboost import XGBClassifier
model = XGBClassifier(scale_pos_weight=estimate)
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_train, Y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

In [ ]:
model.fit(X_train,Y_train)
Y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

As we can see the data is very biased towards a class, we will try to remove the bias by sampling techniques

In [ ]:

data3=data2.copy()

In [ ]:
data2.head()

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
RD=RandomForestClassifier(class_weight={0:estimate,1:1})

In [ ]:
RD.fit(X_train,Y_train)

In [ ]:
Y_pred = RD.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

# SMOTETOMEK Sampling with 0.5 sampling strategy

In [ ]:
from imblearn.combine import SMOTETomek 

In [ ]:
smk = SMOTETomek(sampling_strategy=0.5)


In [ ]:
X_res,Y_res=smk.fit_resample(X,Y)

In [ ]:
Y_res.value_counts()

In [ ]:
Y.value_counts()

In [ ]:
X_train_res,X_test_res,Y_train_res,Y_test_res=train_test_split(X_res,Y_res,test_size=0.3,random_state=7)

In [ ]:
from numpy import mean
from xgboost import XGBClassifier
model = XGBClassifier()
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_train_res, Y_train_res, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

In [ ]:
model.fit(X_train_res,Y_train_res)
Y_pred_res = model.predict(X_test_res)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test_res,Y_pred_res))
print(classification_report(Y_test_res,Y_pred_res))

In [ ]:
accuracy_score(Y_test_res,Y_pred_res)

# SMOTETOMEK Sampling with 0.5 sampling strategy

In [ ]:
smk = SMOTETomek(sampling_strategy=0.8)


In [ ]:
X_res_ety,Y_res_ety=smk.fit_resample(X,Y)

In [ ]:
Y_res_ety.value_counts()

In [ ]:
X_train_ety,X_test_ety,Y_train_ety,Y_test_ety=train_test_split(X_res_ety,Y_res_ety,test_size=0.3,random_state=7)

In [ ]:
from numpy import mean
from xgboost import XGBClassifier
model = XGBClassifier()
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(model, X_train_ety, Y_train_ety, scoring='roc_auc', cv=cv, n_jobs=-1)
# summarize performance
print('Mean ROC AUC: %.5f' % mean(scores))

In [ ]:
model.fit(X_train_ety,Y_train_ety)
Y_pred_ety = model.predict(X_test_ety)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(Y_test_ety,Y_pred_ety))
print(classification_report(Y_test_ety,Y_pred_ety))

In [ ]:
accuracy_score(Y_test_ety,Y_pred_ety)